# Root Cone Lemma for General Linear Groups

Background: This Jupyter Notebook contains code related to my PhD thesis in Mathematics. The thesis is available on the ArXiv: https://arxiv.org/abs/1710.08885. I prove the absolute convergence of the twisted [Arthur-Selberg trace formula](https://en.wikipedia.org/wiki/Arthur%E2%80%93Selberg_trace_formula). The trace formula is an identity of two distributions, namely spectral and geometric. In the geometric side, I needed to prove a conjecture about root systems asked as a MathOverflow question [here](https://mathoverflow.net/questions/259765/technical-lemma-on-root-systems-reduced-to-linear-algebra). 

I wanted to verify the conjecture for the general linear group in small dimensions and did manage to verify it for up to $$n \leq 8.$$ Finally, I did some computation, a change of basis and reduced the question to showing the existence of some linear inequalities. This was later proven [here](https://mathoverflow.net/questions/260545/existence-of-solution-to-these-inequalities). 

In [5]:
import numpy
from numpy.linalg import inv
from fractions import Fraction

n = 6
print("G = SL(%d) \n" % n)
''' Define basis beta and varpi '''
beta = numpy.zeros((n-1,n))
varpi = numpy.zeros((n-1,n))
for i in range(n-1):
    for j in range(n):
        if j == i:
            beta[i,j] = 1
        elif j == i + 1:
            beta[i,j] = -1
        else:
            beta[i,j] = 0
    for j in range(n):
        if j <= i:
            varpi[i,j] = (n-i-1)/float(n)
        else: 
            varpi[i,j] = -(i+1)/float(n)

print("The basis for beta is: ")
print(beta)
print("\nThe basis for varpi is: ")
print(varpi)

def theta(basis):
    '''This function takes an ordered basis and flips the order.'''
    temp = numpy.copy(basis)
    length = len(basis)
    for i in range(length):
        temp[i] = basis[length-1-i]
    return temp

def V_to_a(basis, vector):
    ''' Express vector as a linear combination of basis elements, 
            returns the (n-1) coefficients
            i.e., V to aaa '''
    basis = basis[:,:-1]
    basis = numpy.transpose(inv(basis))
    vector = vector[:-1]
    return numpy.dot(basis,vector)

def last_coordinate(vector):
    ''' This appends the last coordinate to the vector so the sum of all coordinates is zero '''
    return numpy.append(vector, -numpy.sum(vector))

#basis_1 is the original basis and basis_2 is the target basis 
def change_of_basis(basis_1, basis_2):
    ''' This returns the change-of-basis matrix. '''
    '''This is a nice solution actually'''
    A = None
    for i in range(len(basis_1)):
        if i == 0: 
            A = V_to_a(basis_1, basis_2[i])
        else:
            A = numpy.concatenate((A, V_to_a(basis_1, basis_2[i])), axis=0)
    return numpy.reshape(A,(n-1, n-1))

# def w_vector(vector, sigma):
#     ''' Permutes the coordinates of vector according to sigma '''
#     sigma = numpy.append(sigma, sigma[0])
#     new_vector = numpy.copy(vector)
#     for i, item in enumerate(sigma[:-1]):
#         new_vector[item] = vector[sigma[i+1]]
#     return new_vector

def w_vector(mu, sigma):
    ''' Permutes the coordinates of vector in V according to sigma '''
    new_mu = numpy.copy(mu)
    for i, item in enumerate(sigma):
        new_mu[item] = mu[i]
    return new_mu

def w(matrix, sigma):
    ''' Permutes every vector in the matrix according to sigma '''
    new_matrix = numpy.copy(matrix)
    for i in range(len(matrix)):
        new_matrix[i] = w_vector(matrix[i], sigma)
    return new_matrix
        
# vector = (Id - theta.w)(varpi)
# vector = varpi - theta (w(varpi, [0,1]))
# change_of_basis(beta, vector)

def a_to_V(basis, coefficients):
    '''Returns vector in my hyperplane, given coefficients
        i.e., V \to \aaa '''
    vector = numpy.zeros(len(basis)+1)
    for i in range(len(basis)):
        vector = vector + coefficients[i] * numpy.array(basis[i])
    return vector

def mymatrix(coefficients, sigma): # coefficients = rows of identity matrix
    mylambda = a_to_V(varpi, coefficients)
    wlambda = w_vector( mylambda, sigma)
    vector = V_to_a(theta(varpi), wlambda)
    vector = coefficients - vector
    vector = a_to_V(varpi, vector) 
    return V_to_a(beta, vector)

def FL_matrix(coefficients, sigma): # coefficients = rows of identity matrix
    mylambda = a_to_V(varpi, coefficients)
    wlambda = w_vector( mylambda, sigma)
    vector = V_to_a(varpi, wlambda)
    vector = coefficients - vector
    vector = a_to_V(varpi, vector) 
    return V_to_a(beta, vector)

def error_matrix(coefficients, sigma): # coefficients = rows of identity matrix
    mylambda = a_to_V(varpi, coefficients)
    wlambda = w_vector( mylambda, sigma)
    return V_to_a(beta, wlambda) - V_to_a(theta(beta),wlambda)

w = (2,5,4,0,1,3) # w = (sigma(0), sigma(1), ...) really
print("\n---------------------------------")
print("w is ", w)
print("\n---------------------------------")
print("the FL matrix for lambda - w(lambda) is:")
finalmatrix = []
for row in numpy.identity(n-1):
    print(FL_matrix(row, w))
    
print("\n---------------------------------")
print("the error matrix for (1 - theta)(w(lambda)) is:")
finalmatrix = []
for row in numpy.identity(n-1):
    print(error_matrix(row, w))

print("\n---------------------------------")
print("the matrix for lambda - theta.w(lambda) is:")
finalmatrix = []
for row in numpy.identity(n-1):
    print(mymatrix(row, w))

print("\n---------------------------------")


''' Do not forget this: If the output matrix is (a_{i,j}) then coefficient of beta_1 is a_11 c_1 + a_12 c_2 etc. '''

G = SL(6) 

The basis for beta is: 
[[ 1. -1.  0.  0.  0.  0.]
 [ 0.  1. -1.  0.  0.  0.]
 [ 0.  0.  1. -1.  0.  0.]
 [ 0.  0.  0.  1. -1.  0.]
 [ 0.  0.  0.  0.  1. -1.]]

The basis for varpi is: 
[[ 0.83333333 -0.16666667 -0.16666667 -0.16666667 -0.16666667 -0.16666667]
 [ 0.66666667  0.66666667 -0.33333333 -0.33333333 -0.33333333 -0.33333333]
 [ 0.5         0.5         0.5        -0.5        -0.5        -0.5       ]
 [ 0.33333333  0.33333333  0.33333333  0.33333333 -0.66666667 -0.66666667]
 [ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667 -0.83333333]]

---------------------------------
w is  (2, 5, 4, 0, 1, 3)

---------------------------------
the FL matrix for lambda - w(lambda) is:
[  1.00000000e+00   1.00000000e+00   0.00000000e+00   0.00000000e+00
   9.25185854e-18]
[ 1.  2.  1.  1.  1.]
[ 1.  2.  2.  2.  1.]
[ -6.16790569e-17   1.00000000e+00   1.00000000e+00   2.00000000e+00
   1.00000000e+00]
[ -1.11022302e-16  -1.11022302e-16  -5.55111512e-17   1.00000000e+00
 

' Do not forget this: If the output matrix is (a_{i,j}) then coefficient of beta_1 is a_11 c_1 + a_12 c_2 etc. '